<a href="https://colab.research.google.com/github/duybluemind1988/Data-science/blob/master/Sonion/IGI_final_09_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datetime import datetime, timedelta
import seaborn as sns; sns.set()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from plotly.offline import iplot
import pandas as pd
import re
import altair as alt

import statistics as st
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.stats.diagnostic import acorr_ljungbox
import scipy.stats as scs
from math import sqrt
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


#  Prepare data

In [ ]:
#path='/content/drive/My Drive/Data/Sonion/IGI material Membrane all/' # Good
#path='/content/drive/My Drive/Data/Sonion/Armature 3000S/' # Good
#path='/content/drive/My Drive/Data/Sonion/Drive pin E25/' #  good
#path='/content/drive/My Drive/Data/MMD 2019/'
path='/content/drive/My Drive/Data/Sonion/20020-3001979 pj185 contact spring igir 97185002/' #Good

In [ ]:
import glob
#print(path)
all_files1=glob.glob(path + '*.xlsx')
all_files2=glob.glob(path + '*.xlsm')
all_files=all_files1+all_files2
#sort file in directory by reverse:
all_files = sorted(all_files, reverse = False)
print('number of files: ',len(all_files))

number of files:  46


In [ ]:
import os.path, time
print("Last modified: %s" % time.ctime(os.path.getmtime(all_files[-1])))
print("Created: %s" % time.ctime(os.path.getctime(all_files[-1])))
# Se tao 1 file get creation date, chuyen thanh dataframe, sau do chi doc
# nhung file > creattion date nay thoi (6/2016)

Last modified: Tue Aug  4 15:34:00 2020
Created: Thu Aug  6 02:01:20 2020


In [ ]:
material_name=all_files[-1]
material_name=material_name.split('/')[-1].split('.')[0]
material_name

'867975 20020-3001979 pj185 contact spring'

In [ ]:
sheet_all={}
n=0
count_file=1
for filename in all_files:
  #print(filename)
  print('count_file: ',str(count_file) + '/' + str(len(all_files)))
  count_file+=1
  try:
    df = pd.read_excel(filename, index_col = None, header = 0)
    #df.to_excel('file'+str(n)+'.xlsx')
  except:
    print('File cannot be read: '+filename)

  # find begin and end col 
  try:
    # find postion row with Pos/items
    a=df[df.columns[0]].str.contains('Pos.')
    item_row=df[a==True].index[0]
    values_col=df.iloc[item_row,:] # specific row that contain dimension name
    #print(values_col)
    values_col.reset_index(drop=True,inplace=True)
    begin_col=1 # bat dau tu cot thu 1 tro di tu vi tri Pos.\Item
    try:
      end_col=values_col[values_col.isnull() == True].index[0]
    except: # mot so truong hop cot cuoi cung khong phai Nan value thi phai lay cot cuoi cua val_col +1
      end_col=values_col[-1:].index[0]+1
    #print('begin_col',begin_col)
    #print('end_col',end_col)
  except:
    continue

  dim_dict={} # add all value, USL, LSL, UCL... in sheet
  n+=1
  if n==23:n=0
  for name in df.columns[begin_col:end_col]:
    new_df=pd.DataFrame()

    # find begin value collect (check if value is numeric or not):
    value_begin_row=df[df[df.columns[0]].apply(lambda x: isinstance(x, int))].index[0]

    new_df['Value']=df[name][value_begin_row:] #19:
    new_df=new_df.reset_index(drop=True)
    #print(new_df)
    #print(new_df.applymap(np.isreal))
    try:
      non_nummeric_row=new_df[new_df.applymap(np.isreal).values==False].index[0] # App o cuoi dong value 
    except:
      non_nummeric_row=new_df[new_df.isnull().values==True].index[0] # Nan o cuoi dong value
      #continue
    new_df=new_df[:non_nummeric_row]
    new_df['Value']=new_df['Value'].astype('float32')
    
    #print(df)
    # Check Date: (find row and column contains date)
    for i in range(7,10):
      try:
        row_date=df[df[df.columns[i]].apply(lambda x: isinstance(x, datetime))].index[0]
        column_date=i
      except:
        continue
    #print(row_date,column_date)
    new_df['Date']=df[df.columns[column_date]][row_date]
    #print(new_df)
    #new_df['Date']=new_df['Date'].apply(lambda x: x.strftime("%Y %m %d %H"))
    new_df['Date']=pd.to_datetime(new_df['Date'])
    new_df['Date']=new_df['Date'].apply(lambda x : x+timedelta(hours=n)) # and hour from 1 to 23 to avoid duplicate
    
    # chuyen vi tri date ra phia ngoai cung, value vao phia trong
    cols = new_df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    new_df=new_df[cols]
    
    # Add USL, LSL, UCL, LCL, Mean:
    new_df['USL']=df[name][item_row+2] # max
    new_df['LSL']=df[name][item_row+3] # min
    #new_df['Nominal']=df[name][item_row+8] # min
    
    #UCL,LCL,nominal: (lấy 25 gia tri gần nhất thôi)
    #sigma=3
    #new_df['UCL']=new_df['Value'].mean() + new_df['Value'].std()*sigma
    #new_df['LCL']=new_df['Value'].mean() - new_df['Value'].std()*sigma
    #new_df['Mean']=new_df['Value'].mean()
    new_df[new_df.columns[1:]]=new_df[new_df.columns[1:]].astype('float32')
    dim_name=df[name][item_row]
    dim_dict[dim_name]=new_df
  sheet_all[filename]=dim_dict

count_file:  1/46
count_file:  2/46
count_file:  3/46
count_file:  4/46
count_file:  5/46
count_file:  6/46
count_file:  7/46
count_file:  8/46
count_file:  9/46
count_file:  10/46
count_file:  11/46
count_file:  12/46
count_file:  13/46
count_file:  14/46
count_file:  15/46
count_file:  16/46
count_file:  17/46
count_file:  18/46
count_file:  19/46
count_file:  20/46
count_file:  21/46
count_file:  22/46
count_file:  23/46
count_file:  24/46
count_file:  25/46
count_file:  26/46
count_file:  27/46
count_file:  28/46
count_file:  29/46
count_file:  30/46
count_file:  31/46
count_file:  32/46
count_file:  33/46
count_file:  34/46
count_file:  35/46
count_file:  36/46
count_file:  37/46
count_file:  38/46
count_file:  39/46
count_file:  40/46
count_file:  41/46
count_file:  42/46
count_file:  43/46
count_file:  44/46
count_file:  45/46
count_file:  46/46


In [ ]:
#sheet_all

# Base file

Base file is the latest excel file contain all latest dim, it will merge data with all dims in old excel file. If old excel file have dim that not occur in present, it will not appear

In [ ]:
base_file=sheet_all[list(sheet_all.keys())[-1]].copy() # lay file cuoi cung
first_dim=list(base_file.keys())[0]
num_sub_sample=len(base_file[first_dim])
print(num_sub_sample)

21


In [ ]:
# concat all process in each weeks based on keys value:
for file_name in list(sheet_all.keys())[:-1]: # ko tính base week nên từ 1: 
  other_file=sheet_all[file_name]
  for dim_name in other_file.keys(): # dict all process
    for dim_name_base in base_file.keys(): # dict all process
        if dim_name_base==dim_name: 
          base_file[dim_name]=pd.concat([base_file[dim_name],other_file[dim_name]])
          base_file[dim_name]=base_file[dim_name].sort_values(by='Date').reset_index(drop=True).dropna() # drop NaN value in row
# drop non numeric dim:
print('before drop non numeric dim: ',base_file.keys())
dim_drop=[]
for name in base_file.keys():
  print(name,len(base_file[name]))
  if len(base_file[name]) == 0:
    print('drop dim: ',name)
    dim_drop.append(name)
for dim in dim_drop:
  base_file.pop(dim,None)
  #base_file=base_file.pop(dim,None)
print('after drop non numeric dim: ',base_file.keys())

before drop non numeric dim:  dict_keys(['pos.1', 'pos.2', 'pos.3', 'pos.4', 'pos.7', 'pos.6', 'pos.10', 'pos.8', 'pos.5', 'pos.9', 'pos.12', 'pos.13'])
pos.1 860
pos.2 860
pos.3 860
pos.4 860
pos.7 60
pos.6 860
pos.10 860
pos.8 60
pos.5 860
pos.9 860
pos.12 843
pos.13 860
after drop non numeric dim:  dict_keys(['pos.1', 'pos.2', 'pos.3', 'pos.4', 'pos.7', 'pos.6', 'pos.10', 'pos.8', 'pos.5', 'pos.9', 'pos.12', 'pos.13'])


# Calculate UCL, LCL, Cp, Cpk, Ppk base on stdev 25 lates subgroup sample

In [ ]:
constants={
    2:1.128,
    3:1.693,
    4:2.059,
    5:2.326,
    6:2.534,
    7:2.704,
    8:2.847, 
    9: 2.970,
    10: 3.078,
    11: 3.173,
    12: 3.258,
    13: 3.336,
    14: 3.407,
    15: 3.472,
    16: 3.532,
    17:3.588,
    18:3.640,
    19:3.689,
    20:3.735,
}

In [ ]:

def process_performance(df,num_sub_sample):
  #print('dim: ',name)
  n=num_sub_sample
  num_sample=n*25
  
  df_temp=df[-num_sample:].reset_index(drop=True)
  #print(len(df))
  #print(len(df_temp))
  usl=df_temp.USL[0]
  lsl=df_temp.LSL[0]
  #m=df_temp.Value.mean()
  m=df_temp.Value[-n:].mean() # only last day 

  #Ppk
  sigma=st.stdev(df_temp.Value)
  Pp = float(usl - lsl) / (6*sigma)
  Ppu = float(usl - m) / (3*sigma)
  Ppl = float(m - lsl) / (3*sigma)
  Ppk = np.min([Ppu, Ppl])
  

  #UCL, LCL, Mean
  k=3
  df['UCL']=df_temp['Value'].mean() + sigma*k
  df['LCL']=df_temp['Value'].mean() - sigma*k
  df['Mean']=df_temp['Value'].mean()

  #Cpk
  
  temp=df_temp.groupby('Date').agg({'Value':['min','max']})
  temp['Range']=temp['Value','max']-temp['Value','min']
  Range=temp['Range'].mean()

  if n <= 20:
    sigma_within = Range/constants[n]
  else:
    sigma_within = Range/constants[20]

  Cp = float(usl - lsl) / (6*sigma_within)
  Cpu = float(usl - m) / (3*sigma_within)
  Cpl = float(m - lsl) / (3*sigma_within)
  Cpk = np.min([Cpu, Cpl])
  #print('Pp:{:.2f} , Ppk: {:.2f}'.format(Pp,Ppk))
  #print('Cp:{:.2f} , Cpk:{:.2f}'.format(Cp,Cpk))
  if np.isnan(usl):
    Cpk=Cpl
    Ppk=Ppl
  elif np.isnan(lsl):
    Cpk=Cpu
    Ppk=Ppu
  else:
    Cpk = np.min([Cpu, Cpl])
    Ppk = np.min([Ppu, Ppl])
  Cp=round(Cp,2)
  Cpk=round(Cpk,2)
  Pp=round(Pp,2)
  Ppk=round(Ppk,2)
  return Cp,Cpk,Pp,Ppk

In [ ]:
process_indicator_dict={}
process_indicator_df=pd.DataFrame(columns=['Dim','Cp','Pp','Cpk','Ppk'])
#process_indicator_df.columns=['Dim','Cp','Cpk','Pb','Ppk']
i=0
for dim_name in list(base_file.keys()):
  print(dim_name)
  df=base_file[dim_name]
  #try: # object column cannot be calculated process indicator (OK, Not OK...) How to remove object colum in the beginning ?
  Cp,Cpk,Pp,Ppk=process_performance(df,num_sub_sample) 
  #except: continue
  process_indicator_dict[dim_name]=[Cp,Pp,Cpk,Ppk]
  process_indicator_df.loc[i]=dim_name,Cp, Pp, Cpk, Ppk
  i+=1

process_indicator_df=process_indicator_df.sort_values(by='Ppk').reset_index(drop=True)

#conver process indicator dict to list name:
name_list=[]
for name in process_indicator_dict.keys():
  dim_name='dim: ' + name + ' Cp: ' + str(process_indicator_dict[name][0]) + ' Pp: '+ \
  str(process_indicator_dict[name][1]) +' Cpk: ' \
  + str(process_indicator_dict[name][2]) +' Ppk: '+ str(process_indicator_dict[name][3]) 
  name_list.append(dim_name)

pos.1
pos.2
pos.3
pos.4
pos.7
pos.6
pos.10
pos.8
pos.5
pos.9
pos.12
pos.13


In [ ]:
process_indicator_df

,Dim,Cp,Pp,Cpk,Ppk
0,pos.9,0.55,0.47,-0.54,-0.47
1,pos.10,0.12,0.05,-0.09,-0.04
2,pos.12,0.40,0.29,0.17,0.12
3,pos.13,0.73,0.57,0.26,0.20
4,pos.3,0.81,0.57,0.45,0.32
5,pos.4,0.89,0.68,0.66,0.51
6,pos.5,0.88,0.75,0.66,0.57
7,pos.1,0.84,0.66,0.77,0.60
8,pos.2,0.83,0.64,0.80,0.62
9,pos.6,0.84,0.72,0.83,0.71


In [ ]:
name_list

['dim: pos.1 Cp: 0.84 Pp: 0.66 Cpk: 0.77 Ppk: 0.6',
 'dim: pos.2 Cp: 0.83 Pp: 0.64 Cpk: 0.8 Ppk: 0.62',
 'dim: pos.3 Cp: 0.81 Pp: 0.57 Cpk: 0.45 Ppk: 0.32',
 'dim: pos.4 Cp: 0.89 Pp: 0.68 Cpk: 0.66 Ppk: 0.51',
 'dim: pos.7 Cp: 3.93 Pp: 2.96 Cpk: 2.87 Ppk: 2.16',
 'dim: pos.6 Cp: 0.84 Pp: 0.72 Cpk: 0.83 Ppk: 0.71',
 'dim: pos.10 Cp: 0.12 Pp: 0.05 Cpk: -0.09 Ppk: -0.04',
 'dim: pos.8 Cp: 2.13 Pp: 2.02 Cpk: 1.4 Ppk: 1.32',
 'dim: pos.5 Cp: 0.88 Pp: 0.75 Cpk: 0.66 Ppk: 0.57',
 'dim: pos.9 Cp: 0.55 Pp: 0.47 Cpk: -0.54 Ppk: -0.47',
 'dim: pos.12 Cp: 0.4 Pp: 0.29 Cpk: 0.17 Ppk: 0.12',
 'dim: pos.13 Cp: 0.73 Pp: 0.57 Cpk: 0.26 Ppk: 0.2']

In [ ]:
def hightlight_price(row):
    ret = ["" for _ in row.index]
    if row.Cpk < limit or row.Ppk < limit:
      ret[row.index.get_loc("Dim")] = "background-color: yellow"
    if row.Cpk < limit:    
      ret[row.index.get_loc("Cpk")] = "background-color: yellow"
    if row.Ppk < limit:  
      ret[row.index.get_loc("Ppk")] = "background-color: yellow"
    return ret

limit = 1.33  # sigma: 4, Yield: 99.99%   
process_indicator_df.style.apply(hightlight_price, axis=1)

,Dim,Cp,Pp,Cpk,Ppk
0,pos.9,0.550000,0.470000,-0.540000,-0.470000
1,pos.10,0.120000,0.050000,-0.090000,-0.040000
2,pos.12,0.400000,0.290000,0.170000,0.120000
3,pos.13,0.730000,0.570000,0.260000,0.200000
4,pos.3,0.810000,0.570000,0.450000,0.320000
5,pos.4,0.890000,0.680000,0.660000,0.510000
6,pos.5,0.880000,0.750000,0.660000,0.570000
7,pos.1,0.840000,0.660000,0.770000,0.600000
8,pos.2,0.830000,0.640000,0.800000,0.620000
9,pos.6,0.840000,0.720000,0.830000,0.710000


# Line chart

In [ ]:
df_dict=base_file
fig = make_subplots(          # Dim name
    rows=len(df_dict), cols=1,
    #shared_xaxes=True, # share same axis
    vertical_spacing=0.05, # adjust spacing between charts
    #column_widths=[0.8, 0.2],
    #subplot_titles=(list(df_dict.keys()))
    subplot_titles=(name_list)
)
i=1
for name in list(df_dict.keys()): #also group
  #print(name)
  df=df_dict[name]
  df=df.sort_values(by=['Date'])
  for a in df.columns[1:]:
    df[a] = df[a].round(decimals=3)
  df=df.reset_index(drop=True)
  df_group=df.groupby('Date').mean()
  #df=df.set_index('Date')
  #if start_date != '' and end_date != '':
  #  df=df[start_date:end_date] 
  #Control chart 1 
  fig.append_trace(go.Scatter(
                          x=df_group.index, y=df_group['Value'],mode='lines+markers',name='Mean ',
                          line=dict( color='#4280F5')
                          ),row=i, col=1)
  #USL, LSL
  fig.append_trace(go.Scatter(x=df_group.index, y=df_group['USL'],name='USL ', line=dict( color='#FF5733'),mode='lines'),row=i, col=1)
  fig.append_trace(go.Scatter(x=df_group.index, y=df_group['LSL'],name='LSL ',line=dict( color='#FF5733'),mode='lines'),row=i, col=1)
  #fig.append_trace(go.Scatter(x=df['Datef'], y=df['Nominal'],name='Nominal '+name,line=dict( color='#FF5733')),row=i, col=1)
  # UCL, LCL
  fig.append_trace(go.Scatter(x=df_group.index, y=df_group['UCL'],name='UCL ', line=dict( color='#33C2FF'),mode='lines'),row=i, col=1)
  fig.append_trace(go.Scatter(x=df_group.index, y=df_group['LCL'],name='LCL ', line=dict( color='#33C2FF'),mode='lines'),row=i, col=1)
  fig.append_trace(go.Scatter(x=df_group.index, y=df_group['Mean'],name='Mean ', line=dict( color='#33C2FF'),mode='lines'),row=i, col=1)
  i=i+1


fig.update_layout(height=200*len(df_dict), width=1200, title_text=material_name)
#fig update each process (contain a lot of dim inside)
fig.show()
 

# Box plot chart

In [ ]:
df_dict=base_file
fig = make_subplots(          # Dim name
    rows=len(df_dict), cols=1,
    #shared_xaxes=True, # share same axis
    vertical_spacing=0.05, # adjust spacing between charts
    #column_widths=[0.8, 0.2],
    #subplot_titles=(list(df_dict.keys()))
    subplot_titles=(name_list)
)
i=1
for name in list(df_dict.keys()): #also group
  df=df_dict[name]
  df=df.sort_values(by=['Date'])
  for a in df.columns[1:]:
    df[a] = df[a].round(decimals=3)
  df=df.reset_index(drop=True)
  df=df.set_index('Date')
  #df=df.set_index('Date')
  #if start_date != '' and end_date != '':
  #  df=df[start_date:end_date]
  #Control chart 1 
  fig.append_trace(go.Box(
                          x=df.index, y=df['Value'],name='value '+name,
                          line=dict( color='#4280F5')
                          ),row=i, col=1)
  #USL, LSL
  fig.append_trace(go.Scatter(x=df.index, y=df['USL'],name='USL '+name, line=dict( color='#FF5733'),mode='lines'),row=i, col=1)
  fig.append_trace(go.Scatter(x=df.index, y=df['LSL'],name='LSL '+name,line=dict( color='#FF5733'),mode='lines'),row=i, col=1)
  #fig.append_trace(go.Scatter(x=df['Datef'], y=df['Nominal'],name='Nominal '+name,line=dict( color='#FF5733')),row=i, col=1)
  i=i+1


fig.update_layout(height=200*len(df_dict), width=1200, title_text=material_name)
#fig update each process (contain a lot of dim inside)
fig.show()
 

# Forecast

In [ ]:
from fbprophet import Prophet
predict_future=3 # Month

In [ ]:

#@st.cache(suppress_st_warning=True)
#def predict_prophet(DFdict_final):
DFdict_final=base_file
DF_predict_final_={} # only contail predict
#DF_predict_all_final={} # contain predict and past value
for name in DFdict_final.keys(): #also group
  print(name)
  df_temp_2=DFdict_final[name] # gia tri mean thong thuong
  df_temp_2=df_temp_2.groupby('Date').mean()
  df_predict=df_temp_2.copy()
  #df_predict=df_predict.resample('1M').ffill() # gia tri mean da resample
  df_predict=df_predict.resample('M').mean().dropna()
  df_fb=df_predict.reset_index()#
  df_fb=df_fb[[df_fb.columns[0],df_fb.columns[1]]]
  df_fb.columns=['ds','y']

  # Prophet will by default fit weekly and yearly seasonalities,
  model = Prophet( #yearly_seasonality=True
                ) #instantiate Prophet with only yearly seasonality as our data is monthly 
  model.fit(df_fb) #fit the model with your dataframe

  # predict for five months in the furure and MS - month start is the frequency
  future = model.make_future_dataframe(periods = predict_future, freq = 'M')  
  # now lets make the forecasts
  forecast = model.predict(future)
  new_forecast=forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
  new_forecast['USL']=df_temp_2['USL'][0]
  new_forecast['LSL']=df_temp_2['LSL'][0]
  #new_forecast['Nominal']=df_temp_2['Mean'][0]
  new_forecast['UCL']=df_temp_2['UCL'][0]
  new_forecast['LCL']=df_temp_2['LCL'][0]
  new_forecast['mean']=df_temp_2['Mean'][0]
  DF_predict_final_[name]=new_forecast 

C-Pitch


INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


J


INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


I


INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


O


INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Z


In [ ]:
#@st.cache(suppress_st_warning=True,allow_output_mutation=True)
#def line_chart_prophet(DFdict_final,DF_predict_final):

DFdict_final=base_file
DF_predict_final=DF_predict_final_
i=1
#Layout
fig = make_subplots(
    rows=len(DFdict_final), cols=1,
    #shared_xaxes=True,
    vertical_spacing=0.05,
    #column_widths=[0.8, 0.2],
    subplot_titles=(name_list)
)

for name in DFdict_final.keys(): #also group
  df=DFdict_final[name].copy()
  df=df.sort_values(by=['Date'])
  for a in df.columns[1:]:
    df[a] = df[a].round(decimals=3)
  df=df.groupby('Date').mean()
  #Control chart 1 
  fig.append_trace(go.Scatter(
                          x=df.index, y=df['Value'],
                          mode='lines+markers',
                          name='mean ' + name,line=dict( color='#4280F5')
                          ),row=i, col=1
                )
  #Predict:
  df_predict=DF_predict_final[name].copy()
  fig.append_trace(go.Scatter(
                      x=df_predict['ds'], y=df_predict['yhat'],
                      mode='lines+markers',
                      name='mean predict ' + name,line=dict( color='#060891')
                      ),row=i, col=1
            )

  #USL, LSL
  fig.append_trace(go.Scatter(x=df_predict['ds'], y=df_predict['USL'],name='USL '+name, line=dict( color='#FF5733'),mode='lines'),row=i, col=1)
  fig.append_trace(go.Scatter(x=df_predict['ds'], y=df_predict['LSL'],name='LSL '+name,line=dict( color='#FF5733'),mode='lines'),row=i, col=1)
  #fig.append_trace(go.Scatter(x=df_predict['ds'], y=df_predict['Nominal'],name='Nominal '+name,line=dict( color='#FF5733')),row=i, col=1)
  # UCL, LCL
  #fig.append_trace(go.Scatter(x=df_predict['ds'], y=df_predict['UCL'],name='UCL '+name, line=dict( color='#33C2FF')),row=i, col=1)
  #fig.append_trace(go.Scatter(x=df_predict['ds'], y=df_predict['LCL'],name='LCL '+name, line=dict( color='#33C2FF')),row=i, col=1)
  #fig.append_trace(go.Scatter(x=df_predict['ds'], y=df_predict['mean'],name='mean '+name, line=dict( color='#33C2FF')),row=i, col=1)
  # Fill between:
  fig.append_trace(go.Scatter(x=df_predict['ds'], y=df_predict['yhat_upper'],name='yhat_upper '+name, line=dict( color='#9FA0F8'),fill=None,mode='lines'),row=i, col=1)
  fig.append_trace(go.Scatter(x=df_predict['ds'], y=df_predict['yhat_lower'],name='yhat_lower '+name, line=dict( color='#9FA0F8'), fill='tonexty',mode='lines'),row=i, col=1)


  #Final layout:
  #fig.update_layout(height=400, width=1400, title_text='Line chart'+name)
  i=i+1
  
fig.update_layout(height=200*len(DFdict_final), width=1200, title_text=material_name)
fig.show()


#Debug

## Debug value coi, end coil

In [ ]:
#filename_1='/content/drive/My Drive/Data/Sonion/20020-3001979 pj185 contact spring igir 97185002/036176 20020-3001979 pj185 contact spring.xlsx'
filename_1='/content/drive/My Drive/Data/Sonion/20020-3001979 pj185 contact spring igir 97185002/037547 20020-3001979 pj185 contact spring.xlsm'
#7/9/2020
df = pd.read_excel(filename_1, index_col = None, header = 0)
#df.to_excel('contact_sprint_2019.xlsx')
 # find postion row with Pos/items
a=df[df.columns[0]].str.contains('Pos.')
item_row=df[a==True].index[0]
print(item_row)
values_col=df.iloc[item_row,:] # specific row that contain dimension name
print(values_col)
values_col.reset_index(drop=True,inplace=True)
begin_col=1 # bat dau tu cot thu 1 tro di tu vi tri Pos.\Item
end_col=values_col[values_col.isnull() == True].index[0]
print('begin_col',begin_col)
print('end_col',end_col)

7
Unnamed: 0              Pos.\Item
Unnamed: 1                  pos.1
Unnamed: 2                  pos.2
Unnamed: 3                  pos.3
Unnamed: 4                  pos.4
Unnamed: 5                  pos.7
Unnamed: 6                  pos.6
Unnamed: 7                 pos.10
Unnamed: 8                  pos.8
Unnamed: 9                  pos.5
Unnamed: 10                 pos.9
Unnamed: 11                pos.12
Unnamed: 12                pos.13
Unnamed: 13                   NaN
Name: 7, dtype: object
begin_col 1
end_col 13


In [ ]:
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Part name:,NaN,NaN,Item-configuration:,NaN,Drawing no.:,NaN,Control Date:,NaN,ID no.:,NaN,NaN,NaN,NaN
2,Contact spring,NaN,NaN,20020-3001979,NaN,97185002,NaN,2020-07-09 00:00:00,NaN,37547,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Supplier:,OJD,NaN,Batch:,20908,Amount:,NaN,30000,pcs,Inspector:,tnguy,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Measurement/Inspection Items,NaN,NaN,NaN,NaN,Item X® - Measure if be required,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Pos.\Item,pos.1,pos.2,pos.3,pos.4,pos.7,pos.6,pos.10,pos.8,pos.5,pos.9,pos.12,pos.13,NaN
8,Unit,mm,mm,mm,mm,mm,mm,mm,mm,mm,mm,mm,mm,NaN
9,USL,1.9,1.9,1.56,1.17,0.12,0.8,89,0.12,0.8,89,95,95,NaN


In [ ]:
filename_1='/content/drive/My Drive/Data/Sonion/20020-3001979 pj185 contact spring igir 97185002/036176 20020-3001979 pj185 contact spring.xlsx'
#28/11/2019
#filename_1='/content/drive/My Drive/Data/Sonion/20020-3001979 pj185 contact spring igir 97185002/037547 20020-3001979 pj185 contact spring.xlsm'
df = pd.read_excel(filename_1, index_col = None, header = 0)
#df.to_excel('contact_sprint_2019.xlsx')
 # find postion row with Pos/items
a=df[df.columns[0]].str.contains('Pos.')
item_row=df[a==True].index[0]
print(item_row)
values_col=df.iloc[item_row,:] # specific row that contain dimension name
print(values_col)
values_col.reset_index(drop=True,inplace=True)
begin_col=1 # bat dau tu cot thu 1 tro di tu vi tri Pos.\Item
try:
  end_col=values_col[values_col.isnull() == True].index[0]
except:
  end_col=values_col[-1:].index[0]+1
print('begin_col',begin_col)
print('end_col',end_col)

7
Unnamed: 0              Pos.\Item
Unnamed: 1                  pos.1
Unnamed: 2                  pos.2
Unnamed: 3                  pos.3
Unnamed: 4                  pos.4
Unnamed: 5                  pos.7
Unnamed: 6                  pos.6
Unnamed: 7                 pos.10
Unnamed: 8                  pos.8
Unnamed: 9                  pos.5
Unnamed: 10                 pos.9
Unnamed: 11                pos.12
Unnamed: 12                pos.13
Name: 7, dtype: object
begin_col 1
end_col 13


In [ ]:
values_col[-1:].index[0]

12

In [ ]:
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Part name:,NaN,NaN,Item-configuration:,NaN,Drawing no.:,NaN,Control Date:,NaN,ID no.:,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Contact spring,NaN,NaN,20020-3001979,NaN,97185002,NaN,2019-11-28 00:00:00,NaN,36176,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Supplier:,Sored,NaN,Batch:,20908,Amount:,NaN,24700,pcs,Inspector:,tnguy,NaN,NaN
6,Measurement/Inspection Items,NaN,NaN,NaN,NaN,Item X® - Measure if be required,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Pos.\Item,pos.1,pos.2,pos.3,pos.4,pos.7,pos.6,pos.10,pos.8,pos.5,pos.9,pos.12,pos.13
8,Unit,mm,mm,mm,mm,mm,mm,mm,mm,mm,mm,mm,mm
9,USL,1.9,1.9,1.56,1.17,0.12,0.8,89,0.12,0.8,89,95,95


In [ ]:
filename_1='/content/drive/My Drive/Data/Sonion/20020-3001979 pj185 contact spring igir 97185002/027302 20020-3001979 pj185 contact spring.xlsx'
#24/1/2017 Nhieu dim mat khong ro nguyen nhan
#filename_1='/content/drive/My Drive/Data/Sonion/20020-3001979 pj185 contact spring igir 97185002/037547 20020-3001979 pj185 contact spring.xlsm'
df = pd.read_excel(filename_1, index_col = None, header = 0)
#df.to_excel('contact_sprint_2019.xlsx')
 # find postion row with Pos/items
a=df[df.columns[0]].str.contains('Pos.')
item_row=df[a==True].index[0]
print(item_row)
values_col=df.iloc[item_row,:] # specific row that contain dimension name
print(values_col)
values_col.reset_index(drop=True,inplace=True)
begin_col=1 # bat dau tu cot thu 1 tro di tu vi tri Pos.\Item
try:
  end_col=values_col[values_col.isnull() == True].index[0]
except:
  end_col=values_col[-1:].index[0]+1
print('begin_col',begin_col)
print('end_col',end_col)

7
Unnamed: 0              Pos.\Item
Unnamed: 1                  pos.1
Unnamed: 2                  pos.2
Unnamed: 3                  pos.3
Unnamed: 4                  pos.4
Unnamed: 5                  pos.7
Unnamed: 6                  pos.6
Unnamed: 7                 pos.10
Unnamed: 8                  pos.8
Unnamed: 9                  pos.5
Unnamed: 10                 pos.9
Unnamed: 11                pos.12
Unnamed: 12                pos.13
Unnamed: 13                pos.14
Name: 7, dtype: object
begin_col 1
end_col 14


In [ ]:
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Part name:,NaN,NaN,Item-configuration:,NaN,Drawing no.:,NaN,Control Date:,NaN,ID no.:,NaN,NaN,NaN,NaN
2,Contact spring,NaN,NaN,20020-3001979,NaN,97185002,NaN,2017-01-24 00:00:00,NaN,27302,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Supplier:,Sored,NaN,Batch:,15844/8334-16,Amount:,NaN,20000,pcs,Inspector:,truc,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Measurement/Inspection Items,NaN,NaN,NaN,NaN,Item X® - Measure if be required,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Pos.\Item,pos.1,pos.2,pos.3,pos.4,pos.7,pos.6,pos.10,pos.8,pos.5,pos.9,pos.12,pos.13,pos.14
8,Unit,mm,mm,mm,mm,mm,mm,mm,mm,mm,mm,mm,mm,mm
9,USL,1.9,1.9,1.56,1.17,0.12,0.8,89,0.12,0.8,89,95,95,0


In [ ]:
dim_dict={} # add all value, USL, LSL, UCL... in sheet
n+=1
if n==23:n=0
for name in df.columns[begin_col:end_col]:
  print(name)
  new_df=pd.DataFrame()

  # find begin value collect (check if value is numeric or not):
  value_begin_row=df[df[df.columns[0]].apply(lambda x: isinstance(x, int))].index[0]

  new_df['Value']=df[name][value_begin_row:] #19:
  new_df=new_df.reset_index(drop=True)
  print(new_df)
  print(new_df.isnull().values)
  a=new_df[new_df.isnull().values==True].index[0]
  print('a\n',a)
  print('a first index:\n',a.iloc[0].index)
  non_nummeric_row=new_df[new_df.applymap(np.isreal).values==False].index[0] # App o cuoi dong value 
  print(non_nummeric_row)
  new_df=new_df[:non_nummeric_row]
  new_df['Value']=new_df['Value'].astype('float32')

Unnamed: 1
     Value
0   1.8804
1   1.8838
2   1.8845
3   1.8819
4   1.8909
5   1.8673
6   1.8852
7   1.8851
8   1.8913
9   1.8827
10   1.885
11   1.882
12   1.867
13   1.891
14   1.885
15   1.869
16   1.875
17   1.872
18    1.88
19   1.884
20     NaN
21     NaN
22     NaN
23     NaN
24     NaN
25     NaN
26     NaN
27     NaN
28     NaN
29     NaN
30     NaN
31     NaN
32     NaN
33     NaN
[[False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]
 [ True]]
a
 20


AttributeError: ignored

In [ ]:
a.index[0]

## End degub

## Debug Cpk, Ppk, USL, LSL

In [ ]:
base_file.keys()

dict_keys(['pos.1', 'pos.2', 'pos.3', 'pos.4', 'pos.7', 'pos.6', 'pos.10', 'pos.8', 'pos.5', 'pos.9', 'pos.12', 'pos.13'])

In [ ]:
base_file['pos.7']

,Date,Value,USL,LSL


In [ ]:
base_file['pos.8']

,Date,Value,USL,LSL


In [ ]:
#print('dim: ',name)
df=base_file['pos.1']
n=num_sub_sample
num_sample=n*25

df_temp=df[-num_sample:].reset_index(drop=True)
#print(len(df))
#print(len(df_temp))
usl=df_temp.USL[0]
lsl=df_temp.LSL[0]
#m=df_temp.Value.mean()
m=df_temp.Value[-n:].mean() # only last day 

#Ppk
sigma=st.stdev(df_temp.Value)
Pp = float(usl - lsl) / (6*sigma)
Ppu = float(usl - m) / (3*sigma)
Ppl = float(m - lsl) / (3*sigma)
Ppk = np.min([Ppu, Ppl])


#UCL, LCL, Mean
k=3
df['UCL']=df_temp['Value'].mean() + sigma*k
df['LCL']=df_temp['Value'].mean() - sigma*k
df['Mean']=df_temp['Value'].mean()

#Cpk

temp=df_temp.groupby('Date').agg({'Value':['min','max']})
temp['Range']=temp['Value','max']-temp['Value','min']
Range=temp['Range'].mean()

if n <= 20:
  sigma_within = Range/constants[n]
else:
  sigma_within = Range/constants[20]

Cp = float(usl - lsl) / (6*sigma_within)
Cpu = float(usl - m) / (3*sigma_within)
Cpl = float(m - lsl) / (3*sigma_within)
Cpk = np.min([Cpu, Cpl])
print('Pp:{:.2f} , Ppk: {:.2f}'.format(Pp,Ppk))
print('Cp:{:.2f} , Cpk:{:.2f}'.format(Cp,Cpk))
Cp=round(Cp,2)
Cpk=round(Cpk,2)
Pp=round(Pp,2)
Ppk=round(Ppk,2)

Pp:0.59 , Ppk: 0.54
Cp:0.72 , Cpk:0.65


## End debug